# 13. Обнаружение аномалий во временных рядах

## Введение

Обнаружение аномалий (anomaly detection) во временных рядах - это критически важная задача в анализе данных, которая находит применение в самых разных областях: от мониторинга промышленного оборудования до обнаружения мошеннических транзакций и выявления сбоев в IT-системах. Аномалия представляет собой наблюдение, которое значительно отличается от общего паттерна данных.

## 1. Теоретические основы

### 1.1 Типы аномалий во временных рядах

**1. Точечные аномалии (Point Anomalies)**
- Отдельные наблюдения, которые значительно отличаются от остальных
- Пример: резкий скачок температуры из-за неисправности датчика

**2. Контекстуальные аномалии (Contextual Anomalies)**
- Значения, аномальные в определенном контексте
- Пример: высокая температура зимой или низкая летом

**3. Коллективные аномалии (Collective Anomalies)**
- Последовательность наблюдений, которые вместе образуют аномальный паттерн
- Пример: постепенная деградация оборудования

### 1.2 Основные подходы к обнаружению аномалий

**1. Статистические методы**
- Z-score (стандартизированные остатки)
- Межквартильный размах (IQR)
- Тесты на выбросы (Grubbs, Dixon)

**2. Методы машинного обучения**
- Isolation Forest
- Local Outlier Factor (LOF)
- One-Class SVM
- Autoencoder-based methods

**3. Методы на основе прогнозирования**
- ARIMA residuals
- LSTM prediction errors
- Prophet anomaly detection

**4. Методы декомпозиции**
- STL (Seasonal and Trend decomposition using Loess)
- Twitter's AnomalyDetection

### 1.3 Метрики качества

Для оценки качества обнаружения аномалий используются:

- **Precision (Точность)**: $\frac{TP}{TP + FP}$ - доля правильно обнаруженных аномалий среди всех обнаруженных
- **Recall (Полнота)**: $\frac{TP}{TP + FN}$ - доля обнаруженных аномалий среди всех реальных аномалий
- **F1-Score**: $2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}$ - гармоническое среднее precision и recall
- **ROC-AUC**: площадь под ROC-кривой

где:
- TP (True Positive) - правильно обнаруженные аномалии
- FP (False Positive) - ложные срабатывания
- FN (False Negative) - пропущенные аномалии

## 2. Практическая реализация

### 2.1 Подготовка данных и генерация аномалий

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 6)

np.random.seed(42)

In [ ]:
# Генерация синтетического временного ряда с аномалиями
def generate_ts_with_anomalies(n_points=1000, anomaly_ratio=0.05):
    """
    Генерация временного ряда с различными типами аномалий
    
    Parameters:
    -----------
    n_points : int
        Количество точек
    anomaly_ratio : float
        Доля аномалий в данных
    """
    time = np.arange(n_points)
    
    # Основные компоненты ряда
    trend = 0.01 * time
    seasonality = 5 * np.sin(2 * np.pi * time / 50) + 3 * np.sin(2 * np.pi * time / 100)
    noise = np.random.randn(n_points) * 0.5
    
    # Нормальные значения
    ts = trend + seasonality + noise + 20
    
    # Добавление аномалий
    n_anomalies = int(n_points * anomaly_ratio)
    anomaly_indices = np.random.choice(n_points, n_anomalies, replace=False)
    
    # Метки: 1 - нормальное значение, -1 - аномалия
    labels = np.ones(n_points)
    labels[anomaly_indices] = -1
    
    # Типы аномалий
    for idx in anomaly_indices:
        anomaly_type = np.random.choice(['spike', 'drop', 'shift'])
        
        if anomaly_type == 'spike':
            # Резкий скачок вверх
            ts[idx] += np.random.uniform(10, 20)
        elif anomaly_type == 'drop':
            # Резкое падение
            ts[idx] -= np.random.uniform(10, 20)
        elif anomaly_type == 'shift' and idx < n_points - 10:
            # Сдвиг уровня
            shift_length = min(10, n_points - idx)
            ts[idx:idx + shift_length] += np.random.uniform(5, 10)
    
    df = pd.DataFrame({
        'time': time,
        'value': ts,
        'is_anomaly': (labels == -1).astype(int)
    })
    
    return df

# Генерация данных
df = generate_ts_with_anomalies(n_points=1000, anomaly_ratio=0.05)

# Визуализация
fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(df['time'], df['value'], label='Временной ряд', alpha=0.7, linewidth=1)
anomalies = df[df['is_anomaly'] == 1]
ax.scatter(anomalies['time'], anomalies['value'], 
           color='red', s=100, label='Аномалии', zorder=5, alpha=0.7)
ax.set_xlabel('Время')
ax.set_ylabel('Значение')
ax.set_title('Временной ряд с аномалиями')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Всего точек: {len(df)}")
print(f"Количество аномалий: {df['is_anomaly'].sum()}")
print(f"Процент аномалий: {df['is_anomaly'].mean() * 100:.2f}%")

## 2.2 Статистические методы

### 2.2.1 Z-score метод

In [ ]:
def detect_anomalies_zscore(data, threshold=3):
    """
    Обнаружение аномалий методом Z-score
    
    Parameters:
    -----------
    data : array-like
        Временной ряд
    threshold : float
        Порог для Z-score (обычно 2-3)
    
    Returns:
    --------
    anomalies : array
        Булев массив с метками аномалий
    z_scores : array
        Z-scores для каждого наблюдения
    """
    mean = np.mean(data)
    std = np.std(data)
    z_scores = np.abs((data - mean) / std)
    anomalies = z_scores > threshold
    return anomalies, z_scores

# Применение Z-score метода
anomalies_zscore, z_scores = detect_anomalies_zscore(df['value'].values, threshold=3)

# Визуализация
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# График временного ряда с обнаруженными аномалиями
axes[0].plot(df['time'], df['value'], label='Временной ряд', alpha=0.7)
detected = df[anomalies_zscore]
axes[0].scatter(detected['time'], detected['value'], 
                color='red', s=100, label='Обнаруженные аномалии (Z-score)', zorder=5)
true_anomalies = df[df['is_anomaly'] == 1]
axes[0].scatter(true_anomalies['time'], true_anomalies['value'], 
                color='orange', s=50, marker='x', label='Истинные аномалии', zorder=4)
axes[0].set_xlabel('Время')
axes[0].set_ylabel('Значение')
axes[0].set_title('Обнаружение аномалий методом Z-score')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# График Z-scores
axes[1].plot(df['time'], z_scores, label='Z-scores', color='blue', alpha=0.7)
axes[1].axhline(y=3, color='red', linestyle='--', label='Порог (threshold=3)')
axes[1].fill_between(df['time'], 0, z_scores, where=anomalies_zscore, 
                      color='red', alpha=0.3, label='Аномальная зона')
axes[1].set_xlabel('Время')
axes[1].set_ylabel('Z-score')
axes[1].set_title('Z-scores временного ряда')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Оценка качества
from sklearn.metrics import precision_score, recall_score, f1_score

y_true = df['is_anomaly'].values
y_pred = anomalies_zscore.astype(int)

print("\nМетрики качества Z-score метода:")
print(f"Precision: {precision_score(y_true, y_pred):.3f}")
print(f"Recall: {recall_score(y_true, y_pred):.3f}")
print(f"F1-Score: {f1_score(y_true, y_pred):.3f}")
print(f"\nОбнаружено аномалий: {y_pred.sum()}")
print(f"Истинных аномалий: {y_true.sum()}")

### 2.2.2 IQR (Interquartile Range) метод

In [ ]:
def detect_anomalies_iqr(data, k=1.5):
    """
    Обнаружение аномалий методом IQR
    
    Parameters:
    -----------
    data : array-like
        Временной ряд
    k : float
        Коэффициент для расчета границ (обычно 1.5)
    """
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - k * IQR
    upper_bound = Q3 + k * IQR
    
    anomalies = (data < lower_bound) | (data > upper_bound)
    
    return anomalies, lower_bound, upper_bound

# Применение IQR метода
anomalies_iqr, lower_bound, upper_bound = detect_anomalies_iqr(df['value'].values, k=1.5)

# Визуализация
plt.figure(figsize=(16, 6))
plt.plot(df['time'], df['value'], label='Временной ряд', alpha=0.7)
plt.axhline(y=lower_bound, color='green', linestyle='--', label=f'Нижняя граница ({lower_bound:.2f})')
plt.axhline(y=upper_bound, color='green', linestyle='--', label=f'Верхняя граница ({upper_bound:.2f})')
plt.fill_between(df['time'], lower_bound, upper_bound, alpha=0.2, color='green', label='Нормальный диапазон')

detected_iqr = df[anomalies_iqr]
plt.scatter(detected_iqr['time'], detected_iqr['value'], 
            color='red', s=100, label='Обнаруженные аномалии (IQR)', zorder=5)

plt.xlabel('Время')
plt.ylabel('Значение')
plt.title('Обнаружение аномалий методом IQR')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Метрики
y_pred_iqr = anomalies_iqr.astype(int)
print("\nМетрики качества IQR метода:")
print(f"Precision: {precision_score(y_true, y_pred_iqr):.3f}")
print(f"Recall: {recall_score(y_true, y_pred_iqr):.3f}")
print(f"F1-Score: {f1_score(y_true, y_pred_iqr):.3f}")

### 2.2.3 Скользящее среднее и стандартное отклонение

In [ ]:
def detect_anomalies_moving_average(data, window=20, n_std=2):
    """
    Обнаружение аномалий на основе скользящего среднего
    
    Parameters:
    -----------
    data : array-like
        Временной ряд
    window : int
        Размер окна для скользящего среднего
    n_std : float
        Количество стандартных отклонений для порога
    """
    rolling_mean = pd.Series(data).rolling(window=window, center=True).mean()
    rolling_std = pd.Series(data).rolling(window=window, center=True).std()
    
    upper_bound = rolling_mean + n_std * rolling_std
    lower_bound = rolling_mean - n_std * rolling_std
    
    anomalies = (data > upper_bound) | (data < lower_bound)
    
    return anomalies, rolling_mean, upper_bound, lower_bound

# Применение метода скользящего среднего
anomalies_ma, rolling_mean, upper_bound_ma, lower_bound_ma = \
    detect_anomalies_moving_average(df['value'].values, window=50, n_std=2.5)

# Визуализация
plt.figure(figsize=(16, 6))
plt.plot(df['time'], df['value'], label='Временной ряд', alpha=0.6, linewidth=1)
plt.plot(df['time'], rolling_mean, label='Скользящее среднее', color='blue', linewidth=2)
plt.plot(df['time'], upper_bound_ma, '--', label='Верхняя граница', color='green', linewidth=1.5)
plt.plot(df['time'], lower_bound_ma, '--', label='Нижняя граница', color='green', linewidth=1.5)
plt.fill_between(df['time'], lower_bound_ma, upper_bound_ma, alpha=0.2, color='green')

detected_ma = df[anomalies_ma]
plt.scatter(detected_ma['time'], detected_ma['value'], 
            color='red', s=100, label='Обнаруженные аномалии', zorder=5)

plt.xlabel('Время')
plt.ylabel('Значение')
plt.title('Обнаружение аномалий методом скользящего среднего')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Метрики
y_pred_ma = anomalies_ma.astype(int)
print("\nМетрики качества метода скользящего среднего:")
print(f"Precision: {precision_score(y_true, y_pred_ma):.3f}")
print(f"Recall: {recall_score(y_true, y_pred_ma):.3f}")
print(f"F1-Score: {f1_score(y_true, y_pred_ma):.3f}")

## 2.3 Методы машинного обучения

### 2.3.1 Isolation Forest

In [ ]:
# Подготовка признаков
def create_features(df, window=10):
    """
    Создание признаков для ML моделей
    """
    features = pd.DataFrame()
    features['value'] = df['value']
    
    # Скользящие статистики
    features['rolling_mean'] = df['value'].rolling(window=window, center=True).mean()
    features['rolling_std'] = df['value'].rolling(window=window, center=True).std()
    features['rolling_min'] = df['value'].rolling(window=window, center=True).min()
    features['rolling_max'] = df['value'].rolling(window=window, center=True).max()
    
    # Разности
    features['diff_1'] = df['value'].diff(1)
    features['diff_2'] = df['value'].diff(2)
    
    # Лаги
    features['lag_1'] = df['value'].shift(1)
    features['lag_2'] = df['value'].shift(2)
    
    # Заполнение пропусков
    features = features.fillna(method='bfill').fillna(method='ffill')
    
    return features

# Создание признаков
X = create_features(df, window=20)

# Нормализация
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Isolation Forest
iso_forest = IsolationForest(
    contamination=0.05,  # ожидаемая доля аномалий
    random_state=42,
    n_estimators=100
)

# Предсказание: -1 для аномалий, 1 для нормальных точек
predictions_if = iso_forest.fit_predict(X_scaled)
anomaly_scores_if = iso_forest.score_samples(X_scaled)

# Визуализация
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# График временного ряда
axes[0].plot(df['time'], df['value'], label='Временной ряд', alpha=0.7)
anomalies_if = df[predictions_if == -1]
axes[0].scatter(anomalies_if['time'], anomalies_if['value'], 
                color='red', s=100, label='Обнаруженные аномалии (Isolation Forest)', zorder=5)
true_anomalies = df[df['is_anomaly'] == 1]
axes[0].scatter(true_anomalies['time'], true_anomalies['value'], 
                color='orange', s=50, marker='x', label='Истинные аномалии', zorder=4)
axes[0].set_xlabel('Время')
axes[0].set_ylabel('Значение')
axes[0].set_title('Обнаружение аномалий с помощью Isolation Forest')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# График аномальных скоров
axes[1].plot(df['time'], anomaly_scores_if, label='Anomaly Score', color='purple', alpha=0.7)
axes[1].fill_between(df['time'], anomaly_scores_if, 
                     where=(predictions_if == -1), color='red', alpha=0.3, label='Аномалии')
axes[1].set_xlabel('Время')
axes[1].set_ylabel('Anomaly Score')
axes[1].set_title('Anomaly Scores от Isolation Forest (более отрицательные = более аномальные)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Метрики
y_pred_if = (predictions_if == -1).astype(int)
print("\nМетрики качества Isolation Forest:")
print(f"Precision: {precision_score(y_true, y_pred_if):.3f}")
print(f"Recall: {recall_score(y_true, y_pred_if):.3f}")
print(f"F1-Score: {f1_score(y_true, y_pred_if):.3f}")

### 2.3.2 Local Outlier Factor (LOF)

In [ ]:
# Local Outlier Factor
lof = LocalOutlierFactor(
    n_neighbors=20,
    contamination=0.05,
    novelty=False
)

predictions_lof = lof.fit_predict(X_scaled)
anomaly_scores_lof = lof.negative_outlier_factor_

# Визуализация
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

axes[0].plot(df['time'], df['value'], label='Временной ряд', alpha=0.7)
anomalies_lof = df[predictions_lof == -1]
axes[0].scatter(anomalies_lof['time'], anomalies_lof['value'], 
                color='red', s=100, label='Обнаруженные аномалии (LOF)', zorder=5)
axes[0].scatter(true_anomalies['time'], true_anomalies['value'], 
                color='orange', s=50, marker='x', label='Истинные аномалии', zorder=4)
axes[0].set_xlabel('Время')
axes[0].set_ylabel('Значение')
axes[0].set_title('Обнаружение аномалий с помощью Local Outlier Factor')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(df['time'], anomaly_scores_lof, label='LOF Score', color='purple', alpha=0.7)
axes[1].fill_between(df['time'], anomaly_scores_lof, 
                     where=(predictions_lof == -1), color='red', alpha=0.3, label='Аномалии')
axes[1].set_xlabel('Время')
axes[1].set_ylabel('LOF Score')
axes[1].set_title('LOF Scores (более отрицательные = более аномальные)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Метрики
y_pred_lof = (predictions_lof == -1).astype(int)
print("\nМетрики качества LOF:")
print(f"Precision: {precision_score(y_true, y_pred_lof):.3f}")
print(f"Recall: {recall_score(y_true, y_pred_lof):.3f}")
print(f"F1-Score: {f1_score(y_true, y_pred_lof):.3f}")

### 2.3.3 One-Class SVM

In [ ]:
# One-Class SVM
oc_svm = OneClassSVM(
    kernel='rbf',
    gamma='auto',
    nu=0.05  # верхняя граница доли аномалий
)

predictions_svm = oc_svm.fit_predict(X_scaled)
anomaly_scores_svm = oc_svm.score_samples(X_scaled)

# Визуализация
plt.figure(figsize=(16, 6))
plt.plot(df['time'], df['value'], label='Временной ряд', alpha=0.7)
anomalies_svm = df[predictions_svm == -1]
plt.scatter(anomalies_svm['time'], anomalies_svm['value'], 
            color='red', s=100, label='Обнаруженные аномалии (One-Class SVM)', zorder=5)
plt.scatter(true_anomalies['time'], true_anomalies['value'], 
            color='orange', s=50, marker='x', label='Истинные аномалии', zorder=4)
plt.xlabel('Время')
plt.ylabel('Значение')
plt.title('Обнаружение аномалий с помощью One-Class SVM')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Метрики
y_pred_svm = (predictions_svm == -1).astype(int)
print("\nМетрики качества One-Class SVM:")
print(f"Precision: {precision_score(y_true, y_pred_svm):.3f}")
print(f"Recall: {recall_score(y_true, y_pred_svm):.3f}")
print(f"F1-Score: {f1_score(y_true, y_pred_svm):.3f}")

## 2.4 Методы на основе прогнозирования

### 2.4.1 ARIMA residuals

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Обучение ARIMA модели
model_arima = ARIMA(df['value'], order=(2, 1, 2))
fitted_arima = model_arima.fit()

# Предсказания и остатки
predictions_arima = fitted_arima.fittedvalues
residuals = df['value'] - predictions_arima

# Обнаружение аномалий по остаткам (используем Z-score)
threshold_residuals = 3
residuals_zscore = np.abs((residuals - residuals.mean()) / residuals.std())
anomalies_arima = residuals_zscore > threshold_residuals

# Визуализация
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# Исходные данные и предсказания
axes[0].plot(df['time'], df['value'], label='Исходные данные', alpha=0.7)
axes[0].plot(df['time'], predictions_arima, label='ARIMA предсказания', alpha=0.7, linestyle='--')
detected_arima = df[anomalies_arima]
axes[0].scatter(detected_arima['time'], detected_arima['value'], 
                color='red', s=100, label='Обнаруженные аномалии', zorder=5)
axes[0].set_xlabel('Время')
axes[0].set_ylabel('Значение')
axes[0].set_title('ARIMA модель и обнаруженные аномалии')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Остатки
axes[1].plot(df['time'], residuals, label='Остатки', alpha=0.7, color='green')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=1)
axes[1].fill_between(df['time'], residuals, where=anomalies_arima, 
                     color='red', alpha=0.3, label='Аномальные остатки')
axes[1].set_xlabel('Время')
axes[1].set_ylabel('Остатки')
axes[1].set_title('Остатки ARIMA модели')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Z-scores остатков
axes[2].plot(df['time'], residuals_zscore, label='Z-score остатков', alpha=0.7, color='purple')
axes[2].axhline(y=threshold_residuals, color='red', linestyle='--', label=f'Порог ({threshold_residuals})')
axes[2].fill_between(df['time'], 0, residuals_zscore, where=anomalies_arima, 
                     color='red', alpha=0.3, label='Аномальная зона')
axes[2].set_xlabel('Время')
axes[2].set_ylabel('Z-score')
axes[2].set_title('Z-scores остатков')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Метрики
y_pred_arima = anomalies_arima.astype(int)
print("\nМетрики качества метода ARIMA остатков:")
print(f"Precision: {precision_score(y_true, y_pred_arima):.3f}")
print(f"Recall: {recall_score(y_true, y_pred_arima):.3f}")
print(f"F1-Score: {f1_score(y_true, y_pred_arima):.3f}")

## 2.5 Сравнение всех методов

In [ ]:
# Сводная таблица результатов
results = pd.DataFrame({
    'Метод': ['Z-score', 'IQR', 'Moving Average', 'Isolation Forest', 'LOF', 'One-Class SVM', 'ARIMA Residuals'],
    'Precision': [
        precision_score(y_true, y_pred),
        precision_score(y_true, y_pred_iqr),
        precision_score(y_true, y_pred_ma),
        precision_score(y_true, y_pred_if),
        precision_score(y_true, y_pred_lof),
        precision_score(y_true, y_pred_svm),
        precision_score(y_true, y_pred_arima)
    ],
    'Recall': [
        recall_score(y_true, y_pred),
        recall_score(y_true, y_pred_iqr),
        recall_score(y_true, y_pred_ma),
        recall_score(y_true, y_pred_if),
        recall_score(y_true, y_pred_lof),
        recall_score(y_true, y_pred_svm),
        recall_score(y_true, y_pred_arima)
    ],
    'F1-Score': [
        f1_score(y_true, y_pred),
        f1_score(y_true, y_pred_iqr),
        f1_score(y_true, y_pred_ma),
        f1_score(y_true, y_pred_if),
        f1_score(y_true, y_pred_lof),
        f1_score(y_true, y_pred_svm),
        f1_score(y_true, y_pred_arima)
    ]
})

results = results.round(3)
print("\n" + "="*70)
print("СРАВНЕНИЕ ВСЕХ МЕТОДОВ ОБНАРУЖЕНИЯ АНОМАЛИЙ")
print("="*70)
print(results.to_string(index=False))
print("="*70)

# Визуализация сравнения
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ['Precision', 'Recall', 'F1-Score']
for idx, metric in enumerate(metrics):
    axes[idx].barh(results['Метод'], results[metric], color='skyblue', edgecolor='black')
    axes[idx].set_xlabel(metric)
    axes[idx].set_title(f'Сравнение методов по {metric}')
    axes[idx].set_xlim(0, 1)
    axes[idx].grid(axis='x', alpha=0.3)
    
    # Добавление значений на графике
    for i, v in enumerate(results[metric]):
        axes[idx].text(v + 0.02, i, f'{v:.3f}', va='center')

plt.tight_layout()
plt.show()

# Определение лучшего метода
best_method = results.loc[results['F1-Score'].idxmax()]
print(f"\n🏆 Лучший метод по F1-Score: {best_method['Метод']} (F1={best_method['F1-Score']:.3f})")

## 3. Продвинутые техники

### 3.1 Autoencoder для обнаружения аномалий

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Подготовка данных
X_train = X_scaled[df['is_anomaly'] == 0]  # обучаем только на нормальных данных

# Построение Autoencoder
input_dim = X_scaled.shape[1]
encoding_dim = 4  # размер сжатого представления

# Encoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(8, activation='relu')(input_layer)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# Decoder
decoded = Dense(8, activation='relu')(encoded)
decoded = Dense(input_dim, activation='linear')(decoded)

# Autoencoder модель
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Обучение
history_ae = autoencoder.fit(
    X_train, X_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=0
)

# Предсказание и вычисление ошибки реконструкции
X_pred = autoencoder.predict(X_scaled)
reconstruction_error = np.mean(np.abs(X_scaled - X_pred), axis=1)

# Определение порога (например, 95-й перцентиль)
threshold_ae = np.percentile(reconstruction_error, 95)
anomalies_ae = reconstruction_error > threshold_ae

# Визуализация
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# График временного ряда
axes[0].plot(df['time'], df['value'], label='Временной ряд', alpha=0.7)
detected_ae = df[anomalies_ae]
axes[0].scatter(detected_ae['time'], detected_ae['value'], 
                color='red', s=100, label='Обнаруженные аномалии (Autoencoder)', zorder=5)
axes[0].scatter(true_anomalies['time'], true_anomalies['value'], 
                color='orange', s=50, marker='x', label='Истинные аномалии', zorder=4)
axes[0].set_xlabel('Время')
axes[0].set_ylabel('Значение')
axes[0].set_title('Обнаружение аномалий с помощью Autoencoder')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# График ошибки реконструкции
axes[1].plot(df['time'], reconstruction_error, label='Ошибка реконструкции', alpha=0.7, color='blue')
axes[1].axhline(y=threshold_ae, color='red', linestyle='--', label=f'Порог ({threshold_ae:.3f})')
axes[1].fill_between(df['time'], 0, reconstruction_error, where=anomalies_ae, 
                     color='red', alpha=0.3, label='Аномальная зона')
axes[1].set_xlabel('Время')
axes[1].set_ylabel('Reconstruction Error')
axes[1].set_title('Ошибка реконструкции Autoencoder')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Метрики
y_pred_ae = anomalies_ae.astype(int)
print("\nМетрики качества Autoencoder:")
print(f"Precision: {precision_score(y_true, y_pred_ae):.3f}")
print(f"Recall: {recall_score(y_true, y_pred_ae):.3f}")
print(f"F1-Score: {f1_score(y_true, y_pred_ae):.3f}")

# График обучения
plt.figure(figsize=(10, 5))
plt.plot(history_ae.history['loss'], label='Training Loss')
plt.plot(history_ae.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Обучение Autoencoder')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 3.2 Ансамбль методов

In [ ]:
# Создание ансамбля из нескольких методов
# Используем голосование (voting)

# Собираем предсказания всех методов
ensemble_predictions = np.column_stack([
    y_pred,
    y_pred_iqr,
    y_pred_ma,
    y_pred_if,
    y_pred_lof,
    y_pred_svm,
    y_pred_arima
])

# Простое голосование: считаем аномалией, если большинство методов считают аномалией
threshold_voting = 3  # минимум 3 из 7 методов должны обнаружить аномалию
y_pred_ensemble = (ensemble_predictions.sum(axis=1) >= threshold_voting).astype(int)

# Визуализация
plt.figure(figsize=(16, 6))
plt.plot(df['time'], df['value'], label='Временной ряд', alpha=0.7)
detected_ensemble = df[y_pred_ensemble == 1]
plt.scatter(detected_ensemble['time'], detected_ensemble['value'], 
            color='red', s=100, label=f'Обнаруженные аномалии (Ансамбль, порог={threshold_voting})', zorder=5)
plt.scatter(true_anomalies['time'], true_anomalies['value'], 
            color='orange', s=50, marker='x', label='Истинные аномалии', zorder=4)
plt.xlabel('Время')
plt.ylabel('Значение')
plt.title('Обнаружение аномалий с помощью ансамбля методов')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Метрики
print("\nМетрики качества Ансамбля:")
print(f"Precision: {precision_score(y_true, y_pred_ensemble):.3f}")
print(f"Recall: {recall_score(y_true, y_pred_ensemble):.3f}")
print(f"F1-Score: {f1_score(y_true, y_pred_ensemble):.3f}")

# Тепловая карта голосов методов
plt.figure(figsize=(16, 8))
method_names = ['Z-score', 'IQR', 'Moving Avg', 'Iso Forest', 'LOF', 'One-Class SVM', 'ARIMA']
sns.heatmap(ensemble_predictions.T, 
            cmap='RdYlGn_r', 
            yticklabels=method_names,
            cbar_kws={'label': 'Аномалия (1) / Норма (0)'},
            vmin=0, vmax=1)
plt.xlabel('Временные точки')
plt.ylabel('Методы')
plt.title('Голосование методов обнаружения аномалий')
plt.tight_layout()
plt.show()

## 4. Практические рекомендации

### 4.1 Выбор метода в зависимости от задачи

**Статистические методы (Z-score, IQR, Moving Average):**
- ✅ Простота и скорость
- ✅ Интерпретируемость
- ✅ Хорошо работают на простых данных
- ❌ Чувствительны к выбросам
- ❌ Предполагают нормальное распределение
- **Применение:** мониторинг простых метрик, быстрая первичная проверка

**Isolation Forest:**
- ✅ Эффективен для многомерных данных
- ✅ Не требует нормальности распределения
- ✅ Быстрое обучение
- ❌ Может давать ложные срабатывания
- **Применение:** обнаружение аномалий в многомерных данных, когда скорость важна

**LOF (Local Outlier Factor):**
- ✅ Учитывает локальную плотность
- ✅ Хорошо находит локальные аномалии
- ❌ Чувствителен к выбору k (числа соседей)
- ❌ Медленнее на больших данных
- **Применение:** когда аномалии имеют локальный характер

**One-Class SVM:**
- ✅ Работает с высокоразмерными данными
- ✅ Гибкость через выбор ядра
- ❌ Требует настройки гиперпараметров
- ❌ Медленное обучение на больших данных
- **Применение:** когда есть четкая граница между нормальными и аномальными данными

**ARIMA Residuals:**
- ✅ Учитывает временную структуру
- ✅ Хорошо для прогнозируемых рядов
- ❌ Требует стационарности
- ❌ Сложность настройки модели
- **Применение:** временные ряды с трендом и сезонностью

**Autoencoder:**
- ✅ Автоматическое извлечение признаков
- ✅ Эффективен для сложных паттернов
- ❌ Требует больше данных
- ❌ Менее интерпретируем
- **Применение:** сложные многомерные временные ряды, когда много нормальных данных

**Ансамбль:**
- ✅ Более надежен
- ✅ Уменьшает ложные срабатывания
- ❌ Больше вычислений
- **Применение:** критичные системы, где важна надежность

### 4.2 Обработка дисбаланса классов

Аномалии обычно составляют малую долю данных (1-5%), что создает проблему дисбаланса:

1. **Правильный выбор метрик:** не используйте accuracy, используйте Precision, Recall, F1-Score, ROC-AUC
2. **Настройка contamination:** в методах ML указывайте ожидаемую долю аномалий
3. **Threshold tuning:** подбирайте порог под задачу (больше Precision или Recall?)

### 4.3 Производственные системы (Production)

**Мониторинг качества:**
- Регулярно проверяйте False Positive Rate
- Собирайте обратную связь от пользователей
- Переобучайте модели на новых данных

**Масштабируемость:**
- Для real-time: используйте простые статистические методы или Isolation Forest
- Для batch processing: можно использовать более сложные методы

**Интерпретируемость:**
- Сохраняйте не только метку аномалии, но и score/confidence
- Логируйте причину обнаружения (какой признак аномален)
- Визуализируйте аномалии для аналитиков

## 5. Задания для самостоятельной работы

### Задание 1: Базовое (обязательное)

Используйте реальный датасет (например, данные о потреблении электроэнергии или температуры) и:
1. Примените минимум 3 различных метода обнаружения аномалий
2. Сравните результаты
3. Проведите визуальный анализ обнаруженных аномалий
4. Выберите наиболее подходящий метод и обоснуйте выбор

**Критерии оценки:**
- Корректная реализация методов
- Качественная визуализация
- Анализ и интерпретация результатов

In [ ]:
# Место для решения задания 1
# TODO: Ваш код здесь

### Задание 2: Продвинутое

Реализуйте систему обнаружения аномалий в режиме реального времени:
1. Создайте функцию, которая принимает новую точку данных и возвращает, является ли она аномалией
2. Реализуйте механизм адаптивного порога (threshold adapts over time)
3. Добавьте логику для обработки различных типов аномалий:
   - Точечные аномалии
   - Контекстуальные аномалии
   - Коллективные аномалии (несколько последовательных точек)
4. Создайте систему алертов с различными уровнями критичности
5. Визуализируйте работу системы в динамике

**Дополнительно:** реализуйте механизм обратной связи для улучшения модели

In [ ]:
# Место для решения задания 2
# TODO: Ваш код здесь

### Задание 3: Исследовательское

Проведите комплексное исследование обнаружения аномалий:
1. Найдите или создайте датасет с известными аномалиями (можете использовать Yahoo S5 или NAB datasets)
2. Реализуйте минимум 7 различных методов обнаружения аномалий
3. Проведите grid search для подбора оптимальных гиперпараметров каждого метода
4. Постройте ROC-кривые для всех методов
5. Реализуйте и оцените несколько стратегий ансамблирования:
   - Голосование большинством
   - Взвешенное голосование
   - Stacking
6. Создайте итоговый отчет с рекомендациями по выбору метода в различных сценариях
7. Проанализируйте вычислительную сложность каждого метода

**Критерии оценки:**
- Полнота исследования
- Качество экспериментов
- Глубина анализа
- Практические рекомендации

In [ ]:
# Место для решения задания 3
# TODO: Ваш код здесь

## Контрольные вопросы

1. Опишите три типа аномалий во временных рядах и приведите примеры.
2. В чем разница между Z-score и IQR методами? Когда использовать каждый?
3. Как работает Isolation Forest? Почему он эффективен для обнаружения аномалий?
4. Объясните концепцию Local Outlier Factor (LOF).
5. Почему Autoencoder может быть эффективен для обнаружения аномалий?
6. Как выбрать оптимальный порог (threshold) для обнаружения аномалий?
7. Почему accuracy не подходит для оценки качества обнаружения аномалий?
8. В чем преимущества ансамблевого подхода?
9. Как адаптировать методы обнаружения аномалий для работы в режиме реального времени?
10. Какие факторы нужно учитывать при выборе метода для production системы?

## Дополнительные материалы

**Статьи:**
- Chandola, V., Banerjee, A., & Kumar, V. (2009). Anomaly detection: A survey. ACM computing surveys (CSUR), 41(3), 1-58.
- Liu, F. T., Ting, K. M., & Zhou, Z. H. (2008). Isolation forest. In ICDM '08.
- Breunig, M. M., et al. (2000). LOF: identifying density-based local outliers.

**Онлайн ресурсы:**
- [Scikit-learn: Outlier Detection](https://scikit-learn.org/stable/modules/outlier_detection.html)
- [Twitter Anomaly Detection](https://github.com/twitter/AnomalyDetection)
- [NAB (Numenta Anomaly Benchmark)](https://github.com/numenta/NAB)

**Датасеты:**
- [Yahoo S5](https://webscope.sandbox.yahoo.com/catalog.php?datatype=s&did=70)
- [NAB Dataset](https://github.com/numenta/NAB/tree/master/data)
- [UCR Time Series Anomaly Archive](https://www.cs.ucr.edu/~eamonn/time_series_data_2018/)

**Книги:**
- Aggarwal, C. C. (2017). Outlier Analysis. Springer.
- Goldstein, M., & Uchida, S. (2016). A Comparative Evaluation of Unsupervised Anomaly Detection Algorithms for Multivariate Data.